In [1]:
import dash
import dash_table
import pandas as pd
import FundamentalAnalysis as fa
from jupyter_dash import JupyterDash
import dash_html_components as html



In [24]:
QTR_LOOKUP_TABLE = {'01': 'Q1',
                    '02': 'Q1',
                    '03': 'Q1',
                    '04': 'Q2',
                    '05': 'Q2',
                    '06': 'Q2',
                   '07': 'Q3', 
                    '08': 'Q3',
                    '09': 'Q3',
                    '10': 'Q4',
                    '11': 'Q4',
                   '12': 'Q4'}

In [22]:
fa_api_key = '0531b0516802b6f762a2fe685575e4f0'
ticker = 'AAPL'
subreddits = ['teslainvestorsclub']

In [4]:
def get_quarterly_dates(df):
    qtr_dates = []
    for idx in df.index:
        split_date = str(idx).split('-')
        qtr_date = split_date[0] + '-' + QTR_LOOKUP_TABLE[split_date[1]]
        qtr_dates.append(qtr_date)
    return qtr_dates

In [26]:
balance_sheet_quarterly = fa.balance_sheet_statement(ticker, fa_api_key, period="quarterly")
cash_flow_statement_quarterly = fa.cash_flow_statement(ticker, fa_api_key, period="quarter")
financial_ratios_quarterly = fa.financial_ratios(ticker, fa_api_key, period="quarter")
financial_statement_growth = fa.financial_statement_growth(ticker, fa_api_key, period="quarter")
key_metrics = fa.key_metrics(ticker, fa_api_key, period="quarter")

roe = financial_ratios_quarterly.loc['returnOnEquity']
gross_profit_margin = financial_ratios_quarterly.loc['grossProfitMargin'] 
operating_profit_margin = financial_ratios_quarterly.loc['operatingProfitMargin'] 
pretaxProfitMargin = financial_ratios_quarterly.loc['pretaxProfitMargin']
netProfitMargin = financial_ratios_quarterly.loc['netProfitMargin']

revenue_growth = financial_statement_growth.loc['revenueGrowth']
gross_profit_growth = financial_statement_growth.loc['grossProfitGrowth']
ebit_growth = financial_statement_growth.loc['ebitgrowth']
quarterly_dates = get_quarterly_dates(roe)
fmtls_df = pd.DataFrame.from_dict({'qtr_dates': quarterly_dates,
                         'roe': roe, 
                         'grossProfitMargin': gross_profit_margin, 
                         'operatingProfitMargin': operating_profit_margin,
                         'pretaxProfitMargin': pretaxProfitMargin,
                         'netProfitMargin': netProfitMargin,
                         'revenueGrowth': revenue_growth,
                         'ebitGrowth': ebit_growth, 
                         'grossProfitGrowth': gross_profit_growth}, orient='index').T

fmtls_df.index = pd.to_datetime(fmtls_df.index)
fmtls_df = fmtls_df.iloc[0:10]
fmtls_df = fmtls_df.sort_index()
fmtls_df.index = fmtls_df['qtr_dates']
fmtls_df.drop(labels=['qtr_dates'], axis=1, inplace=True)
fmtls_df = fmtls_df.round(3)
fmtls_df

,roe,grossProfitMargin,operatingProfitMargin,pretaxProfitMargin,netProfitMargin,revenueGrowth,ebitGrowth,grossProfitGrowth
qtr_dates,,,,,,,,
2020-Q3,0.193958,0.381604,0.228369,0.230316,0.195879,0.083991,0.128638,0.0885802
2020-Q2,0.155682,0.379995,0.219335,0.220106,0.18854,0.0235282,0.0185171,0.0138578
2020-Q1,0.143436,0.383619,0.220414,0.22525,0.192907,-0.364914,-0.497321,-0.364795
2019-Q4,0.248361,0.383548,0.278472,0.282273,0.242172,0.433776,0.636416,0.448484
2019-Q3,0.151247,0.379653,0.243988,0.251827,0.21371,0.190135,0.353517,0.202007
2019-Q2,0.10413,0.375904,0.214537,0.221357,0.18666,-0.0724985,-0.139471,-0.0730489
2019-Q1,0.10921,0.376127,0.231233,0.237749,0.199276,-0.311885,-0.425383,-0.318754
2018-Q4,0.16935,0.379919,0.276907,0.283549,0.236805,0.340382,0.448443,0.32997
2018-Q3,0.131828,0.382893,0.256248,0.261065,0.224563,0.180888,0.277989,0.179374


In [15]:
df = fmtls_df.T
df.columns = df.columns.astype('str')
df.insert(loc=0, column='metrics', value=df.index)

In [16]:
df

qtr_dates,metrics,2020-Q1,2020-Q2,2020-Q3
roe,roe,0.002,0.011,0.023
grossProfitMargin,grossProfitMargin,0.206,0.210,0.235
operatingProfitMargin,operatingProfitMargin,0.047,0.054,0.092
pretaxProfitMargin,pretaxProfitMargin,0.012,0.025,0.063
netProfitMargin,netProfitMargin,0.003,0.017,0.042
revenueGrowth,revenueGrowth,-0.189,0.009,0.453
ebitGrowth,ebitGrowth,-0.212,0.155,1.474
grossProfitGrowth,grossProfitGrowth,-0.113,0.027,0.628


In [17]:
app = JupyterDash(__name__)

In [18]:
app.layout = dash_table.DataTable(
    id='table',
    columns=[{'name': i, 'id':i} for i in df.columns],
    data=df.to_dict('records'))
                      
if __name__ == '__main__':
    app.run_server(mode='inline')